# Case Study 2: The dataset

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Loading the dataset

The dataset we will work on comes from [here](https://www.kaggle.com/udhhav/employeeattrition).

It consists in a single XLS file. When opened with Pandas' `read_excel` and its default arguments, we get the following:

In [ ]:
import pandas as pd

DATA_FILE = "../../data/employee_attrition/Employee_attrition.xlsx"

df = pd.read_excel(DATA_FILE)

In [ ]:
df

This seems to be some sort of dataset documentation. Let's check if there are other spreadsheets:

In [ ]:
efile = pd.io.excel.ExcelFile("../../data/employee_attrition/Employee_attrition.xlsx")
efile

In [ ]:
efile.sheet_names

The data we are interesed in is split between the `Existing employees` and `Employees who have left` tables.

In [ ]:
df_exist = pd.read_excel(efile, sheet_name="Existing employees")

In [ ]:
df_exist

In [ ]:
df_left = pd.read_excel(efile, sheet_name="Employees who have left")

In [ ]:
df_left

The data looks very clean at a glance.

It still would be a lot more convenient to have all employee data in a single dataframe.
To achieve that, we create a new indicator column that indicates if the employee has left or not, and merge the two resulting tables:

In [ ]:
df_attr = pd.concat(
    [df_exist.assign(has_left=0), df_left.assign(has_left=1)]
).reset_index(drop=True)

In [ ]:
df_attr

Let's gather all this in a function:

In [ ]:
def load_dataset(datafile):
    """Load the attrition dataset

    Reads the input excel file and concatenates left and remaining
    employees in the same data frame. Applies a column name correction.

    Parameters
    ----------
    datafile: str
        The path to the input datafile
    """
    with pd.io.excel.ExcelFile(datafile) as efile:
        df_exist = pd.read_excel(efile, sheet_name="Existing employees")
        df_left = pd.read_excel(efile, sheet_name="Employees who have left")

    df_attr = pd.concat([df_exist.assign(has_left=0), df_left.assign(has_left=1)])

    return df_attr.rename(
        columns={"average_montly_hours": "average_monthly_hours"}
    ).reset_index(drop=True)


df_attr = load_dataset(DATA_FILE)

## Univariate Analysis

In [ ]:
df_attr.info()

In [ ]:
df_attr.describe()

Let's see first the number of current vs former employees in the database:

In [ ]:
df_counts = df_attr.has_left.value_counts().rename("count")
print(df_counts)
df_counts.plot.bar()
plt.xlabel("has_left")
_ = plt.ylabel("Count")

We can draw bar plots and histograms of various variables and differentiate between former and current employees:



In [ ]:
import seaborn as sns

_ = sns.histplot(data=df_attr, x="satisfaction_level", hue="has_left")

In [ ]:
_ = sns.histplot(data=df_attr, x="last_evaluation", hue="has_left")

In [ ]:
_ = sns.catplot(
    data=df_attr.astype({"has_left": str}),
    x="number_project",
    hue="has_left",
    kind="count",
)

**Exercise**: do the same for `average_monthly_hours` and `time_spend_company`.

We can also perform the counts manually, and display the numerical values:

In [ ]:
df_pivot = df_attr.pivot_table(
    index="time_spend_company", columns="has_left", aggfunc="size", fill_value=0
)
df_pivot

Stacked bars help seeing in terms of proportions:

In [ ]:
df_pivot.divide(df_pivot.sum(1) / 100.0, axis="rows").reset_index().plot.barh(
    x="time_spend_company", stacked=True
)
plt.legend(loc="upper left")
_ = plt.xlim(0, 100)

Let's make a utility function for plotting such graphs:

In [ ]:
def plot_stacked_percent_bars(
    df, column, hue="has_left", count_col="Emp ID", order=None
):
    df_pivot = df_attr.pivot_table(
        index=column, columns=hue, values=count_col, aggfunc="count", fill_value=0
    )

    if order is not None:
        df_pivot = df_pivot.loc[order]

    df_pivot.divide(df_pivot.sum(1) / 100.0, axis="rows").reset_index().plot.barh(
        x=column, stacked=True
    )
    plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1), title=hue)
    plt.xlim(0, 100)

A copy of that code is provided in the `mll1.plotting` module:

In [ ]:
from mll1 import plotting

In [ ]:
plotting.plot_stacked_percent_bars(df_attr, "time_spend_company")

In [ ]:
_ = sns.catplot(
    data=df_attr.astype({"has_left": str}),
    x="Work_accident",
    hue="has_left",
    kind="count",
)

In [ ]:
plotting.plot_stacked_percent_bars(df_attr, "Work_accident")

In [ ]:
_ = sns.catplot(
    data=df_attr.astype({"has_left": str}),
    x="promotion_last_5years",
    hue="has_left",
    kind="count",
)

In [ ]:
plotting.plot_stacked_percent_bars(df_attr, "promotion_last_5years")

Beware of the plot interpretation. Compare the stacked plot for `Work_accident` and `promotion_last_5years` for example...

In [ ]:
plotting.plot_stacked_percent_bars(df_attr, "dept")

In [ ]:
plotting.plot_stacked_percent_bars(df_attr, "salary", order=["low", "medium", "high"])

## Multivariate analysis

### Pairwise scatter plots

For starters, we can draw pair-wise plots.  We use scatter plots instead of KDE because some of the variables are not continuous.  For clarity and speed we randomly sample 5% of the data:

In [ ]:
import seaborn as sns

In [ ]:
_ = sns.pairplot(
    data=df_attr.drop(columns=["Emp ID"]).sample(frac=0.05),
    #     kind="kde",
    hue="has_left",
)

We can notice that the distributions for `has_left=1` (in orange) seem to be multi-modal. This means there may be several distinct populations of former employees.

### Correlation matrix

Let's compute the correlation between each pair of numerical variables, and plot the resulting matrix as a heat map:

In [ ]:
num_cols = [
    c for c in df_attr.columns if df_attr.dtypes[c] != "object" and c != "Emp ID"
]

plt.figure(figsize=(10, 8))
sns.heatmap(
    df_attr[num_cols].corr(),
    annot=True,
    fmt="0.2f",
    cmap="coolwarm",
    vmin=-1,  # So that 0 is the middle,
)
plt.xticks(rotation=-30, horizontalalignment="left")
_ = plt.tight_layout()

There is a clear anti-correlation between `satisfaction_level` and `has_left`, which is not too surprising. We can also see that `last_evaluation`, `number_project` and `average_monthly_hours` and `time_spent_company` are highly correlated. It should be interesting to reduce the number of those dimensions while keeping the information they contain.

### Principal Component Analysis

In order to decorrelate the variables (and also reduce the number of dimensions so we can do meaningful 2D plots), we will perform a PCA.

The data will be first preprocessed by a StandardScaler, except for the boolean variables that will be kept as-is:

In [ ]:
num_cols = [
    "satisfaction_level",
    "last_evaluation",
    "number_project",
    "average_monthly_hours",
    "time_spend_company",
]

bool_cols = ["promotion_last_5years", "Work_accident", "has_left"]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


pca = Pipeline(
    [
        (
            "std",
            ColumnTransformer(
                [("scaler", StandardScaler(), num_cols)], remainder="passthrough"
            ),
        ),
        ("pca", PCA(n_components=len(num_cols + bool_cols))),
    ]
)
Xtr = pca.fit_transform(df_attr[num_cols + bool_cols])

In [ ]:
plt.plot(pca["pca"].explained_variance_ratio_.cumsum(), "o-")
plt.xlabel("Number of componants")
_ = plt.ylabel("Cumulated explained variance ratio")

Let's plot the density of the data projected on the first two principal axes:

In [ ]:
_ = sns.histplot(data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2")

It looks like there may be several modalities here. To gain some clarity, we can use KDE and differentiate between former and current employees:

In [ ]:
_ = sns.kdeplot(
    data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2", hue="has_left"
)

One can distinguish 3 distinct components in the distribution for the former employees.

Plotting the correlation circle will let us see where the 8 orignal axes end up on the plane of the first 2 PC axis ; (right: the correlation matrix, for reference):

In [ ]:
plt.figure(figsize=(7, 6))
plotting.plot_correlation_circle(pca["pca"], (0, 1), num_cols + bool_cols)
_ = plt.tight_layout()

plt.figure(figsize=(8, 6))
_ = sns.kdeplot(
    data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2", hue="has_left"
)

### Clustering

Looking at a KDE of the PCA a couple cells above, it seemed that 4 distinct modes could be distinguished.

Clustering techniques offer a practical way to explicitely separate data into distinct components. **The K-means algorithm** is the most famous technique of this kind.
K-means is a partitionning technique that aims at separating the data into K groups (clusters), while minimizing the distance between each point and the mean of its cluster (called its centroid).

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=1)
kmeans.fit(Xtr)

We can plot the densities of the resulting clusters, along with their centroids:

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.kdeplot(
    x=Xtr.T[0], y=Xtr.T[1], hue=kmeans.labels_, palette="tab10", label="cluster"
)
plt.xlabel("pc1")
plt.ylabel("pc2")
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color="k")
plt.title("clusters")

plt.subplot(1, 2, 2)
sns.kdeplot(
    data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2", hue="has_left"
)
_ = plt.title("has_left")

The clusters' centers are not totally centered on the visualized components, but we need to keep in mind that there are additional dimensions that are not represented.

Let's count how many employees of each kind belong to each of the clusters:

In [ ]:
_ = sns.catplot(
    data=df_attr.assign(cluster=kmeans.labels_).astype({"has_left": str}),
    x="cluster",
    hue="has_left",
    kind="count",
)

Cluster 3 contains almost only current employees, while the other 3 have mixed content.

To characterize each of the clusters, we can visualize the centroids with a polar plot:

In [ ]:
# keep numerical columns and normalize
df_tmp = df_attr.drop(columns=["Emp ID", "dept", "salary"])
df_tmp = (df_tmp - df_tmp.min()) / (df_tmp.max() - df_tmp.min())

# clusters' mean as Pandas Dataframe (kmeans.centroids are Numpy)
df_group = df_tmp.groupby(kmeans.labels_).mean()

plt.figure(figsize=(8, 8))
plotting.polar_plot(df_group, normalize=False)

### Hierarchical clustering

Another approach to clustering consists in building iteratively a hierarchy of clusters.
This can be done either:

- by agglomeration, starting from the individual samples and grouping them;
- by division, starting from the whole population

We will try the first approach:

In [ ]:
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=4, compute_distances=True)
y_predict = agg.fit_predict(Xtr)

Agglomerative clustering does not have the notion of centroids, in order to retrieve it, we can still compute the centroid as the barycenter of each cluster:

In [ ]:
centroids = pd.DataFrame(Xtr).groupby(y_predict).mean().values

Since the clusering is agglomerative, way clusters are iteratively merged, forms a graph called **dendrogram** which can sometimes help understand the data structure:

In [ ]:
plt.figure(figsize=(12, 4))
plt.title("Hierarchical Clustering Dendrogram")
plotting.plot_dendrogram(agg, truncate_mode="level", p=4)
_ = plt.xlabel("Number of points in node (or index of point if no parenthesis).")

Dendrogram are easier to interpret when the leaves (data points) can be plotted.

We can project the 4 clusters onto the principal components and plot the respective densities, as we did above for the K-means algorithm:

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.kdeplot(x=Xtr.T[0], y=Xtr.T[1], hue=agg.labels_, palette="tab10")
plt.xlabel("pc1")
plt.ylabel("pc2")
plt.scatter(centroids[:, 0], centroids[:, 1], color="k")
_ = plt.title("clusters")

plt.subplot(1, 2, 2)
sns.kdeplot(
    data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]),
    x="pc1",
    y="pc2",
    hue="has_left",
)
_ = plt.title("has_left")

In [ ]:
_ = sns.catplot(
    data=df_attr.assign(cluster=agg.labels_).astype({"has_left": str}),
    x="cluster",
    hue="has_left",
    kind="count",
)

In [ ]:
# keep numerical columns and normalize
df_tmp = df_attr.drop(columns=["Emp ID", "dept", "salary"])
df_tmp = (df_tmp - df_tmp.min()) / (df_tmp.max() - df_tmp.min())

# clusters' mean as Pandas Dataframe (kmeans.centroids are Numpy)
df_group = df_tmp.groupby(kmeans.labels_).mean()

plt.figure(figsize=(8, 8))
plotting.polar_plot(df_group, normalize=False)

### Gaussian mixture models

#### Definition
Mixture models are a kind of intermediate between parametric models and KDE. The pdf of a mixture model at point $\boldsymbol x$ is typically given by:

$$ p(\boldsymbol x) = \sum_{k=1}^K p(k) p(\boldsymbol x|k) $$

Where $K$ is the number of components, $p(k)$ the prior probability of component $k$ and $p(\boldsymbol x | k)$ is the probability of point $\boldsymbol x$ with respect to component $k$.


The most commonly used parametric form for the componants are Gaussian distributions:

$$p(\boldsymbol x | k) = \mathcal G(\boldsymbol x; \boldsymbol \mu_k, \Sigma_k)$$

so the likelihood of a point $\boldsymbol x$ is:

$$ p(\boldsymbol x) = \sum_{k=1}^K \pi_k \mathcal G(\boldsymbol x; \boldsymbol \mu_k, \Sigma_k) $$

where $\pi_k$ is the component weight (prior probability), $\boldsymbol \mu_k$ and $\Sigma_k$ the covariance matrix.

Gaussian mixture models (GMM) are very similar to Gaussian KDE; they can be seen as a generalization of them. The main advantage of GMMs is that, once trained, their evaluation grows linearly with the number of components which is usually much smaller than the number of training points.

The model is trained using an **Expectation Maximization** approach. This optimization process is an iterative algorithm which alternates two phases:

* Computes the "soft" association of each point to the components (expectation)
* Updates the parameters based on this association (maximization)

This algorithm is **not convex**. This means that we are likely find a local optimum only, and the found solution will be **dependent on the initialization**.

#### The `scikit-learn` implementation

In order to deal with this problem, it is possible to restart the algorithm several times and keep the solution with the highest likelihood. The number of restarts is controlled through the `n_init` parameter (default to 1).


In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=4, n_init=5, random_state=1)
gmm.fit(Xtr)

#### Plotting high-dimensional GMMs (marginalization)

It is relatively easy to project a GMM by marginalizing the Gaussian components along the not displayed axes, which simply consists in taking the subpart of interest in the mean vector and the corresponding subblock of the covariance matrix.

Let's consider a Gaussian component made of 2 sub-blocks:

$$\boldsymbol \mu = \left(\matrix{\boldsymbol \mu_1 \\ \boldsymbol \mu_2}\right)$$

$$\Sigma = \left(\matrix{\Sigma_{11} & \Sigma_{12} \\ \Sigma_{21} & \Sigma_{22}}\right)$$

Marginalizing over the block 2 simply consists in taking:

$$\boldsymbol \mu^1 = \boldsymbol \mu_1$$
$$\Sigma^1 = \Sigma_{11}$$

For a GMM, the weights remain unchanged.

We first display the GMM components as ellipses. The radii and the orientation of the ellipse are computed from the eigen-decomposition of the covariance matrix.

In [ ]:
from mll1 import plotting

_ = sns.kdeplot(
    data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2", hue="has_left"
)

plotting.plot_gmm(gmm, [0, 1])

The `GaussianMixture` object also have a `score_samples()` method which computes the log-likelihood (i.e. the log of the PDF) of each point w.r.t. the model.

The `plot_gmm_pdf` function below first projects (marginalizes) the GMM over the chosen coordinates then computes the value of the PDF for a grid in the plane:

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.title("KDE")
sns.kdeplot(data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2")

xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()

plt.subplot(1, 2, 2)
plt.title("GMM")
plotting.plot_gmm_pdf(
    gmm, coords=[0, 1], xrange=(xmin, xmax), yrange=(ymin, ymax), levels=11
)
_ = plt.xlabel("pc1")

When the data structure is not too complicated GMM can be used both as **density estimation** algorithms (using the `score_samples` method) and as clustering algorithm (using the `predict` method):

In [ ]:
clusters = gmm.predict(Xtr)

sns.kdeplot(
    data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]),
    x="pc1",
    y="pc2",
    hue=clusters.astype(str),
    levels=20,
)

**Note**: The overlap is due to projection.

In [ ]:
from collections import Counter

In [ ]:
Counter(clusters)

One should keep in mind that there is no guarantee that the GMM will **tile** the input space.

It is possible that 2 Gaussian components have very close centers for instance. This is especially the case in the presence of **long tail distributions** since the Gaussian distribution is not well adapted to model this kind of data.

To illustrate this, let's take some points sampled from a Laplace distribution:

In [ ]:
from scipy import stats

np.random.seed(1)
x = stats.laplace().rvs(size=1000)

plt.hist(x, bins=100)
_ = plt.ylabel("Count")

Laplace distributions have a prominent mode, but also long tails.

Let's fit a 2-components Gaussian mixture on this distribution:


In [ ]:
gmm = GaussianMixture(n_components=2)
gmm.fit(x[:, None])

xx = np.linspace(x.min(), x.max(), 101)
plt.hist(x, bins=100, density=True, alpha=0.7, label="hist")
plt.plot(xx, np.exp(gmm.score_samples(xx[:, None])), label="GMM")

for i in range(2):
    weight = gmm.weights_[i]
    mean = gmm.means_[i, 0]
    sigma = gmm.covariances_[i, 0, 0]
    print(
        f"Component: {i}; weight: {weight:0.2f}; mean: {mean: 0.2f}; sigma: {sigma:0.2f}"
    )
    plt.plot(xx, weight * stats.norm(mean, sigma).pdf(xx), label=f"Component {i + 1}")
_ = plt.legend()

Without a priori knowledge, finding the appropriate number of components is a difficult task.

Here is an example of a 10 component mixture fitted on our data:

In [ ]:
gmm = GaussianMixture(10, n_init=5, random_state=1).fit(Xtr)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.title("KDE")
sns.kdeplot(data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2")

xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()

plt.subplot(1, 2, 2)
plt.title("GMM")
plotting.plot_gmm_pdf(
    gmm, coords=[0, 1], xrange=(xmin, xmax), yrange=(ymin, ymax), levels=11
)
_ = plt.xlabel("pc1")

We can see that the main features of the point cloud have been captured.

Intuitively we understand that increasing the number of components will allow us to capture more details in the distribution up to some point where we will simply overfit the distribution and capture sampling artifacts:

In [ ]:
gmm = GaussianMixture(50, n_init=5, random_state=1).fit(Xtr)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.title("KDE")
sns.kdeplot(data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2")

xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()

plt.subplot(1, 2, 2)
plt.title("GMM")
plotting.plot_gmm_pdf(
    gmm, coords=[0, 1], xrange=(xmin, xmax), yrange=(ymin, ymax), levels=11
)
_ = plt.xlabel("pc1")

So there should be a trade-off between the quality of the fit and the number of components used to fit the data.

#### The BIC criterion

One of the most often used criterion is the Bayesian Information Criterion:

$$BIC = |\Theta| \log N - 2 \sum_{i=1}^N \log \mathcal L(\boldsymbol x_i; \Theta)$$

where $N$ is the number of data points, $\Theta$ represents the set of parameters and $|\Theta|$ is the number of parameters. The function $\mathcal L(\boldsymbol x_i; \Theta)$ is the likelihood of the point $\boldsymbol x_i$ according to the model parametrized by $\Theta$.

This quantity decreases as the model correctly fits the dataset, but increases with the number of parameters. This allows to find a trade-off between the model flexibility and its simplicity.

**Exercise**: What is the number of parameters of a $K$ components GMM in $D$ dimensions ?

Let's plot the BIC score obtained when fitting a Gaussian mixture on the data with varying number of components.

Here the dataset used is the artificial one, obtained by sampling a Laplace distribution. So we expect to see a better trade-off for 2 components.

In [ ]:
from tqdm.notebook import tqdm
from scipy import stats

np.random.seed(1)
x = stats.laplace().rvs(size=1000)

ns = range(1, 10)
bics = []
for n in tqdm(ns):
    gmm = GaussianMixture(n, n_init=5, random_state=1)
    gmm.fit(x[:, np.newaxis])
    bics.append(gmm.bic(x[:, np.newaxis]))

plt.plot(ns, bics)
plt.ylabel("BIC")
_ = plt.xlabel("K")

Here the BIC criterion clearly selects the 2-components solution. This suggests that maybe a mixture of an other distribution should be more appropriate.

Unfortunately when the data structure is complicated, the BIC criterion may propose too many components. Let's apply the same procedure as before on the attrition dataset:

In [ ]:
ns = [1, 5, 10, 20, 30, 40, 50]
bics = []
for n in tqdm(ns):
    gmm = GaussianMixture(n, n_init=3, random_state=1)
    gmm.fit(Xtr)
    bics.append(gmm.bic(Xtr))

plt.plot(ns, bics)
plt.xlabel("components")
_ = plt.ylabel("BIC")

As we can see the fit time increases with the number of components to fit. Here the BIC seems to have its minimum around 30 components.

One should keep in mind that the **log-likelihood can vary a lot** for the same number of components depending on the **initialization**. So the variation between two BIC points can be smaller that the variation due to the initialization.

In [ ]:
gmm = GaussianMixture(30, n_init=10, random_state=1).fit(Xtr)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.title("KDE")
sns.kdeplot(data=df_attr.assign(pc1=Xtr.T[0], pc2=Xtr.T[1]), x="pc1", y="pc2")

xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()

plt.subplot(1, 2, 2)
plt.title("GMM")
plotting.plot_gmm_pdf(
    gmm, coords=[0, 1], xrange=(xmin, xmax), yrange=(ymin, ymax), levels=11
)
_ = plt.xlabel("pc1")